In [1]:
# TODO
# add in knife loss estimator for better neckin est.

In [58]:
import sys
sys.path.append("../decklizer")
from engine import *

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math
from random import shuffle, choice
import copy

from IPython.display import clear_output
import time
import datetime
import re

from scipy.optimize import linprog
from collections import Counter
import itertools

import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())
import matplotlib.pyplot as plt

Number of processors:  8


In [3]:
buckets = dict()
buckets['A'] = dict()
buckets['A']['w'] = [i+j for i, j in zip([205, 195, 220, 160],
                                         [3, 2, 3, 1])]
buckets['A']['n'] = [3, 2, 3, 1]
buckets['A']['L'] = 17000
buckets['A']['q'] = [776, 557, 470, 17]
buckets['A']['B'] = 4160

buckets['B'] = dict()
buckets['B']['w'] = [i+j for i, j in zip([575, 626, 438, 622, 749, 546],
                                         [10, 11, 9, 11, 14, 10])]
buckets['B']['n'] = [10, 11, 9, 11, 14, 10]
buckets['B']['L'] = 15000
buckets['B']['q'] = [282, 282, 5, 48, 142, 241]
buckets['B']['B'] = 3050

buckets['C'] = dict()
buckets['C']['w'] = [i+j for i, j in zip([622, 749, 800],
                                         [11, 14, 14])]
buckets['C']['n'] = [11, 14, 14]
buckets['C']['L'] = 15000
buckets['C']['q'] = [175, 495, 330]
buckets['C']['B'] = 3050

buckets['D'] = dict()
buckets['D']['w'] = [i+j for i, j in zip([616, 743],
                                         [11, 14])]
buckets['D']['n'] = [11, 14]
buckets['D']['L'] = 15000
buckets['D']['q'] = [850, 150]
buckets['D']['B'] = 3050

buckets['E'] = dict()
buckets['E']['w'] = [1942, 2044, 2146, 2298]
buckets['E']['n'] = [12, 12, 12, 12]
buckets['E']['L'] = 17650
buckets['E']['q'] = [57, 57, 57, 57]
buckets['E']['B'] = 4010


In [4]:
buckets['A']

{'w': [208, 197, 223, 161],
 'n': [3, 2, 3, 1],
 'L': 17000,
 'q': [776, 557, 470, 17],
 'B': 4160}

In [5]:
import random

In [6]:
def deckle(bucket, max_combo, max_patterns, target, edge, goal, verbiose):
    patterns, layout = seed_patterns(bucket['w'], 
                                     [math.ceil(i*target) 
                                      for i in bucket['q']], 
                                     bucket['B']+(edge*2), 
                                     bucket['n'], 
                                     max_combinations=max_combo, 
                                     goal=goal, 
                                     verbiose=verbiose)
    loss, inv, summary = find_optimum(patterns, 
                                      layout, 
                                      bucket['w'], 
                                      [math.ceil(i*target) 
                                       for i in bucket['q']], 
                                      bucket['B']+(edge*2), 
                                      bucket['n'], 
                                      bucket['L'],
                                    max_combinations=max_combo,
                                    max_patterns=max_patterns,
                                    prioritize='time')
    return [loss, int(summary['jumbos'].sum()), 
            inv, summary, max_combo, 
            summary.shape[0], target, edge]

In [7]:
# global params
random.seed(42)
combination_params = [2,3]
pattern_params = [3,4]
production_targets = [1]
edge_trim_allowance = np.arange(1)
goal = 3
verbiose = False

In [16]:
res = []
for i in range(1000):
    bid = str(random.randint(1000,9999))
    buckets[bid] = dict()
    slits = random.randint(1,9)
    B = random.randint(100,10000)
    L = random.randint(10000,100000)
    w = [random.randint(21,B) for i in range(slits)]
    n = [random.randint(1,20) for i in range(slits)]
    q = [random.randint(10,100) for i in range(slits)]
    pattern_params = [slits]

    buckets[bid]['w'] = w
    buckets[bid]['n'] = n
    buckets[bid]['L'] = L
    buckets[bid]['q'] = q
    buckets[bid]['B'] = B

    bucket = buckets[bid]
    
    params = []
    for max_combo in combination_params:
        for max_patterns in pattern_params:
            for target in production_targets:
                for edge in edge_trim_allowance:
                    params.append([bucket, max_combo, max_patterns, target, edge, goal, verbiose])
    
    for param in params:
        out = deckle(param[0],
                param[1],
                param[2],
                param[3],
                param[4],
                param[5],
                param[6])
        out.append(bid)
        res.append(out)

layout pattern: {1837: 1, 1151: 1, 1304: 1, 1073: 1, 2486: 1, 883: 1, 100: 4}
pattern loss: -215.82 %
minimum doffs to fill order: 98
inventory created: {1837: 14, 1151: 74, 1304: 0, 1073: 72, 2486: 82, 883: 59, 100: 293}

28 possible max 2 combinations
24 unique patterns found
total loss:      21.52 % (1.06e+07 sqm)
total inventory: 1.02 % (5.03e+05 sqm)

inventory created: {1837: 0.0, 1151: 1.0, 1304: 0.0, 1073: 2.0, 2486: 0.0, 883: 1.0, 100: 14.0}

layout summary:

loss: 36.85% 	 84.0 x	 {1837: 1, 1151: 0, 1304: 0, 1073: 0, 2486: 0, 883: 0, 100: 0}
loss: 20.27% 	 10.0 x	 {1837: 0, 1151: 2, 1304: 0, 1073: 0, 2486: 0, 883: 0, 100: 0}
loss: 14.25% 	 16.0 x	 {1837: 0, 1151: 0, 1304: 0, 1073: 0, 2486: 1, 883: 0, 100: 0}
loss: 10.00% 	 49.0 x	 {1837: 0, 1151: 0, 1304: 2, 1073: 0, 2486: 0, 883: 0, 100: 0}
loss: 1.30% 	 20.0 x	 {1837: 0, 1151: 0, 1304: 0, 1073: 1, 2486: 0, 883: 2, 100: 0}
loss: 0.92% 	 4.0 x	 {1837: 0, 1151: 0, 1304: 0, 1073: 2, 2486: 0, 883: 0, 100: 7}
loss: 0.17% 	 5.0 x	

4 unique patterns found
4 possible max 3 patterns

total loss:      13.49 % (1.05e+07 sqm)
total inventory: 0.00 % (0.00e+00 sqm)

inventory created: {9454: 0.0, 8852: 0.0, 7815: 0.0}

layout summary:

loss: 21.18% 	 32.0 x	 {9454: 0, 8852: 0, 7815: 1}
loss: 10.75% 	 29.0 x	 {9454: 0, 8852: 1, 7815: 0}
loss: 4.75% 	 19.0 x	 {9454: 1, 8852: 0, 7815: 0}

total jumbos: 80.0 (7.76e+07 sqm)
layout pattern: {9454: 0, 8852: 0, 7815: 1}
pattern loss: 21.18 %

7 possible max 3 combinations
4 unique patterns found
6 possible max 3 patterns

total loss:      13.49 % (1.05e+07 sqm)
total inventory: 0.00 % (0.00e+00 sqm)

inventory created: {9454: 0.0, 8852: 0.0, 7815: 0.0}

layout summary:

loss: 21.18% 	 32.0 x	 {9454: 0, 8852: 0, 7815: 1}
loss: 10.75% 	 29.0 x	 {9454: 0, 8852: 1, 7815: 0}
loss: 4.75% 	 19.0 x	 {9454: 1, 8852: 0, 7815: 0}

total jumbos: 80.0 (7.76e+07 sqm)
layout pattern: {407: 0, 500: 1, 53: 1, 532: 1, 453: 1, 487: 1, 36: 1, 63: 1, 622: 1}
pattern loss: -254.09 %

45 possible ma

32 unique patterns found
total loss:      17.76 % (1.69e+07 sqm)
total inventory: 0.42 % (4.00e+05 sqm)

inventory created: {1606: 0.0, 3113: 0.0, 1545: 3.0, 526: 1.0, 3285: 0.0, 2208: 0.0, 1988: 0.0, 1270: 0.0}

layout summary:

loss: 25.44% 	 26.0 x	 {1606: 0, 3113: 0, 1545: 2, 526: 0, 3285: 0, 2208: 0, 1988: 0, 1270: 0}
loss: 25.39% 	 95.0 x	 {1606: 0, 3113: 1, 1545: 0, 526: 0, 3285: 0, 2208: 0, 1988: 0, 1270: 0}
loss: 23.00% 	 7.0 x	 {1606: 2, 3113: 0, 1545: 0, 526: 0, 3285: 0, 2208: 0, 1988: 0, 1270: 0}
loss: 21.15% 	 62.0 x	 {1606: 0, 3113: 0, 1545: 0, 526: 0, 3285: 1, 2208: 0, 1988: 0, 1270: 0}
loss: 12.37% 	 20.0 x	 {1606: 0, 3113: 0, 1545: 2, 526: 1, 3285: 0, 2208: 0, 1988: 0, 1270: 0}
loss: 8.52% 	 48.0 x	 {1606: 1, 3113: 0, 1545: 0, 526: 0, 3285: 0, 2208: 1, 1988: 0, 1270: 0}
loss: 1.03% 	 6.0 x	 {1606: 0, 3113: 0, 1545: 1, 526: 0, 3285: 0, 2208: 0, 1988: 0, 1270: 2}
loss: 0.45% 	 30.0 x	 {1606: 1, 3113: 0, 1545: 0, 526: 1, 3285: 0, 2208: 0, 1988: 1, 1270: 0}

total jumbos: 

44 unique patterns found
total loss:      11.55 % (2.53e+06 sqm)
total inventory: 0.50 % (1.11e+05 sqm)

inventory created: {219: 0.0, 254: 1.0, 617: 0.0, 106: 0.0, 665: 0.0, 702: 0.0, 221: 1.0, 522: 1.0, 239: 1.0}

layout summary:

loss: 41.08% 	 32.0 x	 {219: 0, 254: 0, 617: 0, 106: 0, 665: 0, 702: 0, 221: 0, 522: 1, 239: 0}
loss: 19.64% 	 22.0 x	 {219: 0, 254: 0, 617: 0, 106: 0, 665: 0, 702: 1, 221: 0, 522: 0, 239: 0}
loss: 18.52% 	 21.0 x	 {219: 0, 254: 0, 617: 0, 106: 0, 665: 0, 702: 0, 221: 0, 522: 0, 239: 3}
loss: 16.05% 	 11.0 x	 {219: 0, 254: 0, 617: 0, 106: 0, 665: 0, 702: 0, 221: 1, 522: 1, 239: 0}
loss: 15.94% 	 15.0 x	 {219: 1, 254: 0, 617: 0, 106: 0, 665: 0, 702: 0, 221: 0, 522: 1, 239: 0}
loss: 10.10% 	 25.0 x	 {219: 0, 254: 3, 617: 0, 106: 0, 665: 0, 702: 0, 221: 0, 522: 0, 239: 0}
loss: 4.38% 	 73.0 x	 {219: 1, 254: 0, 617: 1, 106: 0, 665: 0, 702: 0, 221: 0, 522: 0, 239: 0}
loss: 2.69% 	 17.0 x	 {219: 0, 254: 0, 617: 0, 106: 3, 665: 0, 702: 0, 221: 0, 522: 1, 239: 0}
l

36 unique patterns found
total loss:      9.62 % (2.92e+06 sqm)
total inventory: 0.64 % (1.95e+05 sqm)

inventory created: {3012: 1.0, 5171: 0.0, 193: 1.0, 6377: 0.0, 6505: 0.0, 2781: 0.0, 1962: 1.0, 4040: 1.0, 3259: 1.0}

layout summary:

loss: 40.14% 	 1.0 x	 {3012: 0, 5171: 0, 193: 0, 6377: 0, 6505: 0, 2781: 0, 1962: 0, 4040: 1, 3259: 0}
loss: 23.43% 	 34.0 x	 {3012: 0, 5171: 1, 193: 0, 6377: 0, 6505: 0, 2781: 0, 1962: 0, 4040: 0, 3259: 0}
loss: 17.75% 	 28.0 x	 {3012: 0, 5171: 0, 193: 0, 6377: 0, 6505: 0, 2781: 2, 1962: 0, 4040: 0, 3259: 0}
loss: 11.09% 	 54.0 x	 {3012: 0, 5171: 0, 193: 0, 6377: 0, 6505: 0, 2781: 0, 1962: 1, 4040: 1, 3259: 0}
loss: 10.63% 	 8.0 x	 {3012: 2, 5171: 0, 193: 0, 6377: 0, 6505: 0, 2781: 0, 1962: 0, 4040: 0, 3259: 0}
loss: 5.58% 	 76.0 x	 {3012: 0, 5171: 0, 193: 0, 6377: 1, 6505: 0, 2781: 0, 1962: 0, 4040: 0, 3259: 0}
loss: 3.88% 	 36.0 x	 {3012: 0, 5171: 0, 193: 0, 6377: 0, 6505: 1, 2781: 0, 1962: 0, 4040: 0, 3259: 0}
loss: 3.72% 	 44.0 x	 {3012: 0, 5171

...

41 unique patterns found
total loss:      13.02 % (9.62e+06 sqm)
total inventory: 0.65 % (4.80e+05 sqm)

inventory created: {5548: 0.0, 2297: 0.0, 1253: 1.0, 2809: 2.0, 8574: 0.0, 6250: 1.0, 7101: 0.0, 694: 2.0, 5181: 0.0}

layout summary:

loss: 35.67% 	 74.0 x	 {5548: 1, 2297: 0, 1253: 0, 2809: 0, 8574: 0, 6250: 0, 7101: 0, 694: 0, 5181: 0}
loss: 12.97% 	 25.0 x	 {5548: 0, 2297: 0, 1253: 1, 2809: 0, 8574: 0, 6250: 1, 7101: 0, 694: 0, 5181: 0}
loss: 7.27% 	 29.0 x	 {5548: 0, 2297: 0, 1253: 0, 2809: 1, 8574: 0, 6250: 0, 7101: 0, 694: 0, 5181: 1}
loss: 3.34% 	 13.0 x	 {5548: 0, 2297: 0, 1253: 0, 2809: 0, 8574: 0, 6250: 1, 7101: 0, 694: 3, 5181: 0}
loss: 3.22% 	 14.0 x	 {5548: 0, 2297: 0, 1253: 1, 2809: 0, 8574: 0, 6250: 0, 7101: 1, 694: 0, 5181: 0}
loss: 1.99% 	 12.0 x	 {5548: 0, 2297: 0, 1253: 0, 2809: 3, 8574: 0, 6250: 0, 7101: 0, 694: 0, 5181: 0}
loss: 0.96% 	 30.0 x	 {5548: 0, 2297: 1, 1253: 0, 2809: 0, 8574: 0, 6250: 1, 7101: 0, 694: 0, 5181: 0}
loss: 0.74% 	 15.0 x	 {5548: 0, 2297

29 unique patterns found
total loss:      12.04 % (7.94e+05 sqm)
total inventory: 0.43 % (2.85e+04 sqm)

inventory created: {373: 0.0, 93: 2.0, 413: 0.0, 141: 2.0, 28: 0.0, 394: 0.0, 91: 1.0}

layout summary:

loss: 18.47% 	 84.0 x	 {373: 0, 93: 0, 413: 0, 141: 0, 28: 0, 394: 0, 91: 0}
loss: 16.56% 	 23.0 x	 {373: 1, 93: 0, 413: 0, 141: 0, 28: 0, 394: 0, 91: 0}
loss: 15.92% 	 6.0 x	 {373: 0, 93: 0, 413: 0, 141: 0, 28: 0, 394: 0, 91: 4}
loss: 12.74% 	 47.0 x	 {373: 0, 93: 0, 413: 0, 141: 0, 28: 0, 394: 1, 91: 0}
loss: 9.98% 	 54.0 x	 {373: 0, 93: 0, 413: 1, 141: 0, 28: 0, 394: 0, 91: 0}
loss: 3.82% 	 24.0 x	 {373: 0, 93: 0, 413: 0, 141: 3, 28: 0, 394: 0, 91: 0}
loss: 1.70% 	 23.0 x	 {373: 0, 93: 3, 413: 0, 141: 1, 28: 0, 394: 0, 91: 0}
loss: 1.06% 	 14.0 x	 {373: 0, 93: 0, 413: 1, 141: 0, 28: 1, 394: 0, 91: 0}

total jumbos: 275.0 (6.60e+06 sqm)
layout pattern: {1140: 0, 5903: 0, 6911: 0, 4678: 1, 6569: 1, 7551: 1, 3826: 1, 8648: 1}
pattern loss: -239.06 %

36 possible max 2 combination

8 unique patterns found
total loss:      13.85 % (1.16e+07 sqm)
total inventory: 0.09 % (7.66e+04 sqm)

inventory created: {6870: 0.0, 6626: 0.0, 1639: 1.0, 2687: 0.0}

layout summary:

loss: 29.05% 	 36.0 x	 {6870: 0, 6626: 0, 1639: 0, 2687: 2}
loss: 13.55% 	 17.0 x	 {6870: 0, 6626: 0, 1639: 4, 2687: 0}
loss: 12.48% 	 93.0 x	 {6870: 0, 6626: 1, 1639: 0, 2687: 0}
loss: 9.29% 	 91.0 x	 {6870: 1, 6626: 0, 1639: 0, 2687: 0}

total jumbos: 237.0 (8.40e+07 sqm)
layout pattern: {2617: 0, 7587: 0, 3131: 0, 3005: 1, 4986: 1, 3544: 1, 5409: 1, 5097: 1, 3016: 1}
pattern loss: -228.75 %

45 possible max 2 combinations
25 unique patterns found
total loss:      17.34 % (9.44e+06 sqm)
total inventory: 0.44 % (2.37e+05 sqm)

inventory created: {2617: 1.0, 7587: 0.0, 3131: 1.0, 3005: 1.0, 4986: 0.0, 3544: 1.0, 5409: 0.0, 5097: 0.0, 3016: 0.0}

layout summary:

loss: 33.22% 	 40.0 x	 {2617: 0, 7587: 0, 3131: 0, 3005: 0, 4986: 0, 3544: 0, 5409: 0, 5097: 1, 3016: 0}
loss: 31.18% 	 15.0 x	 {2617: 2, 7587:

19 unique patterns found
total loss:      23.09 % (1.51e+07 sqm)
total inventory: 0.31 % (2.00e+05 sqm)

inventory created: {2655: 0.0, 3437: 0.0, 3271: 0.0, 3222: 0.0, 109: 7.0, 2367: 1.0, 1041: 1.0}

layout summary:

loss: 39.40% 	 76.0 x	 {2655: 0, 3437: 0, 3271: 0, 3222: 0, 109: 0, 2367: 1, 1041: 0}
loss: 32.30% 	 55.0 x	 {2655: 1, 3437: 0, 3271: 0, 3222: 0, 109: 0, 2367: 0, 1041: 0}
loss: 19.99% 	 6.0 x	 {2655: 0, 3437: 0, 3271: 0, 3222: 0, 109: 0, 2367: 0, 1041: 3}
loss: 17.47% 	 50.0 x	 {2655: 0, 3437: 0, 3271: 0, 3222: 1, 109: 0, 2367: 0, 1041: 0}
loss: 16.61% 	 83.0 x	 {2655: 0, 3437: 0, 3271: 1, 3222: 0, 109: 0, 2367: 0, 1041: 0}
loss: 12.30% 	 71.0 x	 {2655: 0, 3437: 1, 3271: 0, 3222: 0, 109: 0, 2367: 0, 1041: 0}
loss: 2.65% 	 7.0 x	 {2655: 0, 3437: 0, 3271: 0, 3222: 0, 109: 13, 2367: 1, 1041: 0}

total jumbos: 348.0 (6.55e+07 sqm)
layout pattern: {7493: 1, 2087: 1, 735: 1, 1127: 1, 6564: 1}
pattern loss: -127.44 %
minimum doffs to fill order: 86
inventory created: {7493: 75

24 unique patterns found
total loss:      11.57 % (4.72e+05 sqm)
total inventory: 0.69 % (2.81e+04 sqm)

inventory created: {1345: 0.0, 450: 1.0, 1425: 0.0, 1141: 0.0, 1163: 0.0, 208: 6.0, 1283: 0.0, 248: 3.0}

layout summary:

loss: 23.44% 	 41.0 x	 {1345: 0, 450: 0, 1425: 0, 1141: 1, 1163: 0, 208: 0, 1283: 0, 248: 0}
loss: 13.80% 	 85.0 x	 {1345: 0, 450: 0, 1425: 0, 1141: 0, 1163: 0, 208: 0, 1283: 1, 248: 0}
loss: 8.77% 	 36.0 x	 {1345: 1, 450: 0, 1425: 0, 1141: 0, 1163: 0, 208: 0, 1283: 0, 248: 0}
loss: 6.36% 	 9.0 x	 {1345: 0, 450: 3, 1425: 0, 1141: 0, 1163: 0, 208: 0, 1283: 0, 248: 0}
loss: 4.22% 	 44.0 x	 {1345: 0, 450: 0, 1425: 0, 1141: 0, 1163: 1, 208: 0, 1283: 0, 248: 1}
loss: 3.88% 	 12.0 x	 {1345: 0, 450: 0, 1425: 1, 1141: 0, 1163: 0, 208: 0, 1283: 0, 248: 0}
loss: 2.01% 	 7.0 x	 {1345: 0, 450: 0, 1425: 0, 1141: 0, 1163: 0, 208: 7, 1283: 0, 248: 0}
loss: 0.20% 	 4.0 x	 {1345: 0, 450: 1, 1425: 0, 1141: 0, 1163: 0, 208: 0, 1283: 0, 248: 4}

total jumbos: 238.0 (4.08e+06 sqm)
l

In [18]:
df = pd.DataFrame(res)

In [20]:
df.columns = ['loss', 'jumbos', 'inventory', 'summary', 'max_combo',
              'layouts', 'target', 'edge', 'pid']



In [43]:
dff = pd.DataFrame(buckets)

In [44]:
dff = dff.T

In [46]:
joined = dff.join(df.set_index('pid'))

In [47]:
joined.shape

(2846, 13)

In [50]:
joined.dropna(0).shape

(2000, 13)

In [52]:
final = joined.dropna(0)

In [61]:
final.to_csv('../decklizer/tests/battery_results.csv', index=True)

In [60]:
final.head()

,w,n,L,q,B,loss,jumbos,inventory,summary,max_combo,layouts,target,edge
1000,"[3392, 3296, 5519, 5353, 7053]","[13, 18, 5, 20, 1]",86487,"[57, 27, 57, 61, 53]",7662,20.474816,214.0,"{3379: 1.0, 3278: 1.0, 5514: 0.0, 5333: 0.0, 7...",loss jumbos ...,2.0,5.0,1.0,0.0
1000,"[3392, 3296, 5519, 5353, 7053]","[13, 18, 5, 20, 1]",86487,"[57, 27, 57, 61, 53]",7662,20.474816,214.0,"{3379: 1.0, 3278: 1.0, 5514: 0.0, 5333: 0.0, 7...",loss jumbos ...,3.0,5.0,1.0,0.0
1001,"[231, 3657, 6163, 7875, 4399]","[14, 5, 8, 11, 3]",66525,"[10, 98, 70, 62, 51]",8183,10.936898,208.0,"{217: 6.0, 3652: 1.0, 6155: 1.0, 7864: 0.0, 43...",loss jumbos ...,2.0,5.0,1.0,0.0
1001,"[231, 3657, 6163, 7875, 4399]","[14, 5, 8, 11, 3]",66525,"[10, 98, 70, 62, 51]",8183,10.936898,208.0,"{217: 6.0, 3652: 1.0, 6155: 1.0, 7864: 0.0, 43...",loss jumbos ...,3.0,5.0,1.0,0.0
1008,"[107, 243, 102]","[1, 3, 20]",43242,"[93, 73, 22]",397,13.332754,87.0,"{106: 0.0, 240: 0.0, 82: 0.0}",loss jumbos layout 0 ...,2.0,3.0,1.0,0.0


In [69]:
pd.DataFrame(final.loc[final['loss'] < -89]['summary'][0])

,loss,jumbos,layout
0,6.63965,3,"{5487: 0, 6402: 0, 588: 10}"
1,-94.7631,59,"{5487: 1, 6402: 1, 588: 1}"


In [86]:
final.loc[final['layouts'].isin([1,2])]

,w,n,L,q,B,loss,jumbos,inventory,summary,max_combo,layouts,target,edge
1032,[301],[17],12382,[58],437,31.121281,58.0,{284: 0.0},loss jumbos layout 0 31.1213 58 ...,2.0,1.0,1.0,0.0
1032,[301],[17],12382,[58],437,31.121281,58.0,{284: 0.0},loss jumbos layout 0 31.1213 58 ...,3.0,1.0,1.0,0.0
1043,[6320],[19],22103,[90],9596,34.139225,90.0,{6301: 0.0},loss jumbos layout 0 34.1392 90...,2.0,1.0,1.0,0.0
1043,[6320],[19],22103,[90],9596,34.139225,90.0,{6301: 0.0},loss jumbos layout 0 34.1392 90...,3.0,1.0,1.0,0.0
1048,[1822],[16],80947,[97],4659,21.785791,49.0,{1806: 1.0},loss jumbos layout 0 21.7858 49...,2.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9941,"[2824, 2895]","[20, 19]",58414,"[75, 86]",3953,27.601177,161.0,"{2804: 0.0, 2876: 0.0}",loss jumbos layout 0 28.56...,2.0,2.0,1.0,0.0
9941,"[2824, 2895]","[20, 19]",58414,"[75, 86]",3953,27.601177,161.0,"{2804: 0.0, 2876: 0.0}",loss jumbos layout 0 28.56...,3.0,2.0,1.0,0.0
9960,"[1444, 279, 4528]","[7, 3, 13]",40939,"[76, 25, 91]",6025,0.924354,91.0,"{1437: 10.0, 276: 0.0, 4515: 0.0}",loss jumbos layout...,3.0,2.0,1.0,0.0
9979,"[1175, 1948]","[9, 20]",18419,"[79, 45]",2408,11.246824,85.0,"{1166: 1.0, 1928: 0.0}",loss jumbos layout 0 19.1...,2.0,2.0,1.0,0.0


In [87]:
from ipywidgets import SelectMultiple, interact

In [111]:
final = final.reset_index()

In [128]:

all(x < final.iloc[0]['B']/2 for x in final.iloc[0]['w'])

False

In [107]:
layouts_selector = SelectMultiple(
    options=list(np.sort(final['layouts'].unique().astype(int))),
    value=list(np.sort(final['layouts'].unique().astype(int)))
)

In [108]:
def hist_plot(layouts=layouts_selector):
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    
    final.loc[final['layouts'].isin(layouts)].plot(kind='hist', y='loss', bins=100, ax=ax)

In [109]:
interact(hist_plot)

interactive(children=(SelectMultiple(description='layouts', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), options=…

<function __main__.hist_plot(layouts=SelectMultiple(description='layouts', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)))>

In [10]:
# [bucket, max_combo, max_patterns, target, edge, goal, verbiose]

patterns, layout = seed_patterns(param[0]['w'], 
                                     param[0]['q'], 
                                     param[0]['B'], 
                                     param[0]['n'], 
                                     max_combinations=param[1], 
                                     goal=param[5], 
                                     verbiose=param[6])

In [11]:
loss, inv, summary = find_optimum(patterns, 
                                      layout, 
                                      param[0]['w'], 
                                      param[0]['q'], 
                                      param[0]['B'], 
                                      param[0]['n'], 
                                      param[0]['L'],
                                    max_combinations=param[1],
                                    max_patterns=param[2],
                                    prioritize='time')

10 possible max 3 patterns

total loss:      18.72 % (1.39e+07 sqm)
total inventory: 0.00 % (0.00e+00 sqm)

inventory created: {4363: 0.0, 2704: 0.0, 3714: 0.0}

layout summary:

loss: 38.92% 	 82.0 x	 {4363: 0, 2704: 1, 3714: 0}
loss: 16.32% 	 37.0 x	 {4363: 0, 2704: 0, 3714: 1}
loss: 2.04% 	 94.0 x	 {4363: 1, 2704: 0, 3714: 0}

total jumbos: 213.0 (7.43e+07 sqm)


In [12]:
bucket

{'w': [4369, 2724, 3732],
 'n': [6, 20, 18],
 'L': 78191,
 'q': [94, 82, 37],
 'B': 4460}

In [13]:
w=bucket['w']
q=bucket['q']
B=bucket['B']
n=bucket['n']
max_combinations=3
goal=3
verbiose=True
'''
creates a number of optimal patterns for deckling

Parameters
----------
w: list
    list of widths (int)
q: list
    list of rolls for each width (int)
B: int
    usuable width per mother roll
n: list
    neck in for each width (int)
max_combinations: int, default 3
    maximum number of unique products (widths) to have on a mother roll
goal: int, default 3
    the desired number of recovered patterns from the knapsack problem
    for every unique grouping of unique widths at max_combinations
verbiose: bool, default True
    turns on/off print statements during execution

Returns
-------
patterns: list of lists
    list of pattern, loss pairs. Pattern is a dictionary containing a width,
    count pair that describes the pattern on the mother roll. Loss is the
    percent material loss for the pattern.
layout: list
    list of counts for every width on the mother roll. Layout is the best
    possible pattern in terms of minimizing mother rolls to create the order
    with a single pattern.
'''
layout = make_best_pattern(q, w, n, B, verbiose=verbiose)
combos = []
for i in range(1,max_combinations+1)[::-1]:
    combos += list(itertools.combinations(w,r=i))
if verbiose:
    print('')
    print("{} possible max {} combinations".format(len(combos),max_combinations))
patterns = []
for combo in combos:
    # due to the naive soln, combos of len 1 should be skipped.

    # knapsack/store_patterns will only find one solution (0?) if
    # the width is over half the length of the bin
    if len(combo) == 1:
        sub_goal = 1
        # if B / combo[0] < 2:
        #     sub_goal = 1
        # else:
        #     sub_goal = goal
    else:
        sub_goal = goal
    # only provide knapsack with relevant variables
    s = []
    for i in combo:
        s += (int(B/i)*[i])
    t = initt(B,len(s))
    knapsack(s, s, B, len(s), t)
    t = np.array(t)
    patterns += store_patterns(t, s, B, goal=sub_goal)
    for j in range(3):
        for i in patterns:
            for key in list(i[0].keys()):
                loss = (B - np.sum(np.array(list(i[0].keys())) *
                    np.array(list(i[0].values())))) - key
                if loss > 0:
                    i[0][key] += 1
                    i[1] = loss
uni_list = []
for i in patterns:
    if i not in uni_list:
        uni_list.append(i)
patterns = uni_list
patterns = list(np.array(patterns)[np.array(patterns)[:,1]>=0])

# the naive patterns should be kept due to their usefullness
# in order fulfilment regardless of loss
naive = init_layouts(B, w)
for i in naive:
    i = [-j for j in i]
    patterns.append([dict(zip(w,i)),0])

if verbiose:
    print("{} unique patterns found".format(len(patterns)))
# return patterns, layout

layout pattern: {4363: 0, 2704: 0, 3714: 1}
pattern loss: 16.32 %

7 possible max 3 combinations
6 unique patterns found


In [14]:
w = param[0]['w'] 
q = param[0]['q']
B = param[0]['B']
n = param[0]['n']
L = param[0]['L']
max_combinations=param[1]
max_patterns=param[2]
prioritize='time'
qt = None
'''
Finds the best possible slitter schedule by linear optimization of a set of
patterns.

Parameters
----------
patterns: list of lists
    list of pattern, loss pairs. Pattern is a dictionary containing a width,
    count pair that describes the pattern on the mother roll. Loss is the
    percent material loss for the pattern.
layout: list
    list of counts for every width on the mother roll. Layout is the best
    possible pattern in terms of minimizing mother rolls to create the order
    with a single pattern.
w: list
    list of widths (int)
q: list
    list of rolls for each width (int)
B: int
    usuable width (mm) of mother roll
n: list
    neck in for each width (int)
L: int
    length (m) of mother roll
max_combinations: int, default 3
    maximum number of unique products (widths) to have on a mother roll
max_patterns: int, default 3
    maximum number of patterns for deckle schedule
prioritize: str, default 'time'
    only relevant when max_patterns < 4. When max_patterns < 4 either the
    lowest material waste linear opimization of patterns is returned
    ('material loss') or the fewest mother rolls used is returned ('time')
qt: list
    default: None. List of the true values for q if algorithm is called
    using an adjusted production target. This is used to set the output
    results inventory to the true inventory created as a result of the
    true order

Returns
-------
loss: float
    percent loss for the deckle schedule
inventory: dictionary
    width, count pairs that describe the inventory created above the order
    critieria
summary: dataframe
    dataframe of loss, jumbos, layout that describe the percent material
    loss, total number of jumbos (or doffs), and the dictionary of width,
    count pairs that describe the pattern for every pattern in the deckle
    schedule
'''
if qt == None:
    qt = q
if prioritize == 'material loss':
    inv_loss = 0
elif prioritize == 'time':
    inv_loss = 1
if 1 < max_patterns < 4:
    # find best of X combination
    if len(w) <= max_combinations:
        pattern_combos = list(itertools.combinations(patterns,r=max_patterns-1))
    else:
        pattern_combos = list(itertools.combinations(patterns,r=max_patterns))
    print("{} possible max {} patterns".format(len(pattern_combos),max_patterns), end='\n\n')
    best_of = []
    for combo in pattern_combos:
        patterns2 = combo
        lhs_ineq = []
        for pattern in patterns2:
            inset = []
            for width in w:
                try:
                    inset.append(-pattern[0][width])
                except:
                    inset.append(0)
            lhs_ineq.append(inset)
    #     naive = init_layouts(B, w)
    #     lhs_ineq = lhs_ineq + naive
        if len(w) <= max_combinations:
            lhs_ineq.append([-i for i in layout])
        lhs_ineq = np.array(lhs_ineq).T.tolist()
        rhs_ineq = [-i for i in q]
        obj = np.ones(len(lhs_ineq[0]))

        result = linprog(c=obj,
                A_ub=lhs_ineq,
                b_ub=rhs_ineq,
                method="revised simplex")
        if result['success'] == True:
            sheet = np.sum([(i*j) for i,j in zip(w, np.array(lhs_ineq))],axis=0)#*np.ceil(result['x'])
            inventory = dict(zip([i-j for i,j in zip(w,n)],np.sum(np.array(lhs_ineq)*-1*\
                                                                  np.ceil(result['x']),axis=1)-np.array(q)))

            # create layout summary
            jumbos = list(np.ceil(result['x'])[np.ceil(result['x'])>0])
            temp = np.array(lhs_ineq)*-1*np.where(np.ceil(result['x']) != 0, 1, 0)
            temp = temp[:, temp.any(0)].T
            non_zero_layouts = list([dict(zip([i-j for i,j in zip(w,n)], i)) for i in temp])

            sheet_loss = [B+i for i in sheet]
            sheet_loss = [i / B * 100 for i,j in zip(sheet_loss,np.where(result['x'] > 0, 1, 0)) if j > 0]

            # remove extra layouts due to ceiling rounding from linprog
            sorted_jumbos = [x for _,x in sorted(zip(sheet_loss,jumbos))][::-1]
            sorted_layouts = np.array(non_zero_layouts)[np.array(sheet_loss).argsort()][::-1]
            sorted_losses = [x for _,x in sorted(zip(sheet_loss,sheet_loss))][::-1]
            for index, layout2 in enumerate(np.array(non_zero_layouts)[np.array(sheet_loss).argsort()][::-1]):
                if all(np.array(list(inventory.values())) - np.array(list(layout2.values())) > 0):
                    sorted_jumbos[index] -= 1
                    new_values = np.array(list(inventory.values())) - np.array(list(layout2.values()))
                    inventory.update(zip(inventory,new_values))

                    # clear layouts that have been set to 0
            summary = (list(zip(sorted_jumbos, sorted_layouts, sorted_losses)))
            summ = []
            for i in summary:
                if i[0] > 0:
                    summ.append(i)
            summary=summ
            loss = sum([i[0]*i[2] for i in summary])/sum([i[0] for i in summary])

            best_of.append([loss, sum(list(inventory.values())), patterns2])

    # minimize inventory or minimize mat. loss
    arr = np.array(best_of, dtype=object)
    patterns_final = arr[np.argmin(arr[:,inv_loss])][2]
elif max_patterns == 1:
    patterns_final = [[dict(zip(w,layout)), 0]]

else:
    patterns_final = patterns

# find overall best combination
# format layouts for linear optimization
lhs_ineq = []
for pattern in patterns_final:
    inset = []
    for width in w:
        try:
            inset.append(-pattern[0][width])
        except:
            inset.append(0)
    lhs_ineq.append(inset)
# naive = init_layouts(B, w)
# lhs_ineq = lhs_ineq + naive

if len(w) <= max_combinations:
    lhs_ineq.append([-i for i in layout])
lhs_ineq = np.array(lhs_ineq).T.tolist()
rhs_ineq = [-i for i in q]
obj = np.ones(len(lhs_ineq[0]))

result = linprog(c=obj,
        A_ub=lhs_ineq,
        b_ub=rhs_ineq,
        method="revised simplex")
if result['success'] == False:
    print('Error')
    print(result['message'])
    return 0


15 possible max 3 patterns



SyntaxError: 'return' outside function (<ipython-input-14-2dc62def3548>, line 169)

In [ ]:
sys.setrecursionlimit(int(1e4))

In [ ]:
bucket

In [ ]:
sys.getrecursionlimit()

In [ ]:
sum(combo) > (B - min(combo))

In [ ]:
sum(combo)

In [ ]:
B

In [ ]:
combo

In [ ]:
t

In [ ]:
patterns

In [ ]:
len(s)

In [ ]:
s

In [ ]:
s = []
for i in combo:
    s += (int(B/i)*[i])
t = initt(B,len(s))
knapsack(s, s, B, len(s), t)
t = np.array(t)

In [ ]:
t

In [ ]:
t.sum(1)

In [ ]:
patterns += store_patterns(t, s, B, goal=3)
for j in range(3):
    for i in patterns:
        for key in list(i[0].keys()):
            loss = (B - np.sum(np.array(list(i[0].keys())) *
                np.array(list(i[0].values())))) - key
            if loss > 0:
                i[0][key] += 1
                i[1] = loss

In [ ]:
reconstruct(N, W, t, s)

In [ ]:
t[-1].argsort()

In [ ]:
t.argsort()

In [ ]:
t.flatten()[t.flatten().argsort()[-1]]

In [ ]:
np.argwhere(t == t.flatten().argsort()[-1])

In [ ]:
pattern

In [ ]:
for pair in np.argwhere(t == 4948):
        checks += 1
        N, W = pair
        sack = reconstruct(N, W, t, s)
        pattern = Counter(np.array(s)[list(sack)])
        loss = B - np.sum(np.array(list(pattern.keys())) *
                        np.array(list(pattern.values())))
        if loss > 0:
            patterns.append([pattern, loss])
        if len(patterns) >= goal:
            break
        elif checks >= cutoff:
            break
        found += 1
        if found > 1:
            break

In [ ]:
patterns = []
bit = 1
checks = 0
goal = 100
while (len(patterns) < goal):
    found = 0
    for pair in np.argwhere(t == t.flatten()[t.flatten().argsort()[-bit]]):
        N, W = pair
        sack = reconstruct(N, W, t, s)
        pattern = Counter(np.array(s)[list(sack)])
        loss = B - np.sum(np.array(list(pattern.keys())) *
                        np.array(list(pattern.values())))
        if loss > 0:
            patterns.append([pattern, loss])
        if len(patterns) >= goal:
            break
        found += 1
        if found > 1:
            break
    bit += 1

In [ ]:
patterns

In [ ]:
def store_patterns(t, s, B, goal=5, cutoff=1e2):
    patterns = []
    bit = 0
    checks = 0
    while (len(patterns) < goal) and (checks < cutoff):
        found = 0
        for pair in np.argwhere(t == t.max()-bit):
            checks += 1
            N, W = pair
            sack = reconstruct(N, W, t, s)
            pattern = Counter(np.array(s)[list(sack)])
            loss = B - np.sum(np.array(list(pattern.keys())) *
                            np.array(list(pattern.values())))
            if loss > 0:
                patterns.append([pattern, loss])
            if len(patterns) >= goal:
                break
            elif checks >= cutoff:
                break
            found += 1
            if found > 1:
                break
        bit += 2
    return patterns

In [ ]:
patterns

In [ ]:
combos

In [ ]:
combos